In [1]:
import random
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

random.seed(1)
np.random.seed(1)
tf.random.set_seed(1)

In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

num_classes = 10
num_features = 784  # x_train.shape[1] * x_train.shape[2]

x_train, x_test = tf.reshape(x_train, [-1, 784]), tf.reshape(x_test, [-1, 784])

# normalization
x_train = x_train / 255.
x_test = x_test / 255.

2022-07-27 13:40:44.238924: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-07-27 13:40:45.338398: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 9970 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:03:00.0, compute capability: 6.1
2022-07-27 13:40:45.339479: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 10402 MB memory:  -> device: 1, name: NVIDIA GeForce GTX 1080 Ti, pci bus id: 0000:04:00.0, compute capability: 6.1


In [7]:
print(x_train)
print(x_test)

tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(60000, 784), dtype=float32)
tf.Tensor(
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]], shape=(10000, 784), dtype=float32)


In [3]:
from tensorflow.keras import Model, layers

# Model Define
class NeuralNet(Model):
    def __init__(self):
        super().__init__()
        
        # MLP layer 이므로 Dense
        self.fc1 = layers.Dense(128, activation=tf.nn.relu)
        self.fc2 = layers.Dense(256, activation=tf.nn.relu)
        self.out = layers.Dense(10) # MNIST 는 10개로 분류
        
    def call(self, x, is_train=False):
        x = self.fc1(x)
        x = self.fc2(x)
        x = self.out(x)
        
        if not is_train:    # Training이 아니면
            x = tf.nn.softmax(x)  # softmax 통과
            
        return x

In [5]:
neural_net = NeuralNet()

In [8]:
def cross_entropy_loss(x, y):
    y = tf.cast(y, tf.int64)
    loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=x)
    return tf.reduce_mean(loss)

In [13]:
lr = 0.001
optimizer = tf.optimizers.SGD(lr)

def run_optimization(x, y):
    with tf.GradientTape() as g:
        pred = neural_net(x, is_train=False)
        loss = cross_entropy_loss(pred, y)
    
    params = neural_net.trainable_variables
    gradients = g.gradient(loss, params)
    optimizer.apply_gradients(zip(gradients, params))
    
def accuracy(y_pred, y_true):
    correct_prediction = tf.equal(tf.argmax(y_pred, axis=1), tf.cast(y_true, tf.int64))
    return tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [15]:
batch_size = 200
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(60000).batch(batch_size).prefetch(1)

epoch = 60
display_epoch = 5

for epo in range(1, epoch + 1):
    for step, (batch_x, batch_y) in enumerate(train_data, 1):
        run_optimization(batch_x, batch_y)
        
    if epo % display_epoch == 0:
        pred = neural_net(batch_x)
        loss = cross_entropy_loss(pred, batch_y)
        acc = accuracy(pred, batch_y)
        print(f"epoch: {epo}, loss: {loss.numpy()}, acc: {acc.numpy()}")

epoch: 5, loss: 2.2741382122039795, acc: 0.1899999976158142
epoch: 10, loss: 2.224592685699463, acc: 0.3400000035762787
epoch: 15, loss: 2.1877450942993164, acc: 0.4050000011920929
epoch: 20, loss: 2.165534734725952, acc: 0.4050000011920929
epoch: 25, loss: 2.1457669734954834, acc: 0.375
epoch: 30, loss: 2.0793464183807373, acc: 0.4350000023841858
epoch: 35, loss: 2.0766611099243164, acc: 0.4099999964237213
epoch: 40, loss: 2.0178754329681396, acc: 0.47999998927116394
epoch: 45, loss: 1.9432852268218994, acc: 0.6399999856948853
epoch: 50, loss: 1.9123320579528809, acc: 0.6850000023841858
epoch: 55, loss: 1.840525507926941, acc: 0.7900000214576721
epoch: 60, loss: 1.8254246711730957, acc: 0.7450000047683716


In [17]:
pred = neural_net(x_test)
print(f"Test Accuracy: {accuracy(pred, y_text).numpy()}")

Test Accuracy: 0.7450000047683716


In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.metrics import SparseCategoricalAccuracy

In [19]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

num_classes = 10
num_features = 784  # x_train.shape[1] * x_train.shape[2]

# x_train, x_test = tf.reshape(x_train, [-1, 784]), tf.reshape(x_test, [-1, 784])

# normalization
x_train = x_train / 255.
x_test = x_test / 255.

In [20]:
layer1 = Flatten(input_shape = (28, 28))
layer2 = Dense(128, activation = 'relu')
layer3 = Dense(128, activation = 'relu')
layer4 = Dense(10)

In [21]:
mlp = Sequential([layer1, layer2, layer3, layer4])

In [25]:
mlp.compile(optimizer = tf.keras.optimizers.SGD(0.01),
            loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics = ['accuracy'])

In [26]:
result = mlp.fit(x_train, y_train, batch_size=32, epochs=10)

Epoch 1/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.6347 - accuracy: 0.8314 - sparse_categorical_crossentropy: 0.0000e+00
Epoch 2/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2952 - accuracy: 0.9151 - sparse_categorical_crossentropy: 0.0000e+00
Epoch 3/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.2436 - accuracy: 0.9296 - sparse_categorical_crossentropy: 0.0000e+00
Epoch 4/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.2098 - accuracy: 0.9399 - sparse_categorical_crossentropy: 0.0000e+00
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1853 - accuracy: 0.9467 - sparse_categorical_crossentropy: 0.0000e+00
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1659 - accuracy: 0.9525 - sparse_categorical_crossentropy: 0.0000e+00
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.1499 - accuracy: 0.9567 - sparse_c

In [27]:
mlp.evaluate(x_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.1208 - accuracy: 0.9631 - sparse_categorical_crossentropy: 0.0000e+00


[0.12076966464519501, 0.963100016117096, 0.0, 0.0]

In [ ]:
plt.plot(result.history['accuracy'])